<h2>Importamos librerías

In [2]:
import sqlite3
import pandas as pd

<h2>Creamos cursor

In [3]:
miConexion = sqlite3.connect("sql-murder-mystery.db")
miCursor = miConexion.cursor()

<h2>Creamos una función para poder hacer las consultas a la base de datos

In [4]:
def sql_query(query):
    
    miCursor.execute(query)
    ans = miCursor.fetchall()
    names = [description[0] for description in miCursor.description]

    return pd.DataFrame(ans,columns=names)

<h2>Filtramos en crime_scene_report con los datos del enunciado

In [5]:
df = sql_query(
'''SELECT * FROM crime_scene_report WHERE 
date = 20180115 AND 
city = "SQL City" AND 
type = "murder"''')

df["description"][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

<h2>Obtenemos la siguiente pista</h2>

Security footage shows that there were 2 witnesses. 
The first witness lives at the last house on 'Northwestern Dr'. 
The second witness, named Annabel, lives somewhere on 'Franklin Ave'.

<h2>Buscamos quién vive en la última casa de "Northwestern Dr":

In [6]:


df = sql_query('''SELECT * FROM person WHERE address_street_name = "Northwestern Dr" ORDER BY address_number ASC''')
df.tail(1)

,id,name,license_id,address_number,address_street_name,ssn
49,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Morty es el que vive en la ultima casa de "Northwestern Dr"

<h2>Tenemos dos sospechosos:</h2>

* Annabel Miller
* Morty Schapiro


In [7]:
df = sql_query('''SELECT * FROM person WHERE address_street_name = "Franklin Ave" AND name ="Annabel Miller"''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [8]:
df = sql_query('''SELECT * FROM person WHERE name ="Morty Schapiro"''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


<h2>¿Qué declaró Annabel?

In [9]:
df = sql_query('''SELECT * FROM interview WHERE person_id = 16371''')
df["transcript"][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

<h2>¿Qué declaró Morty?

In [10]:
df = sql_query('''SELECT * FROM interview WHERE person_id = 14887''')
df["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

<h2>Filtramos con lo que declaro Morty: </h2>

* Miembro "Gold
* Mochila empieza por "48Z" 
* Matrícula incluye "H42W"

In [11]:
df = sql_query('''SELECT * FROM get_fit_now_member WHERE membership_status = "gold" AND id LIKE "48Z%"''')
df

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


<h2>Vemos cuantos coches tienen esa parte de matrícula

In [12]:
df = sql_query('''SELECT * FROM drivers_license WHERE plate_number LIKE "%H42W%"''')
df

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


<h2>Como Morty declaró que vio a un hombre, descartamos a la chica

In [13]:
df[1:3:]

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


<h2>Vemos a quién pertenencen estas matrículas

In [14]:
df = sql_query('''SELECT * FROM person WHERE license_id = 423327''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [15]:
df = sql_query('''SELECT * FROM person WHERE license_id = 664760''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671


<h2>¿Quién de estos dos sospechosos es miembro "Gold"

In [16]:
df = sql_query('''SELECT * FROM get_fit_now_member WHERE person_id = 51739''')
df
#No es miembro, por lo que no pudo llevar esa mochila, descartado

,id,person_id,name,membership_start_date,membership_status


In [17]:
df = sql_query('''SELECT * FROM get_fit_now_member WHERE person_id = 67318''')
df
#Es miebro "Gold"

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


Solo Jeremy es miembro Gold


In [ ]:
solution = (0, 'Jeremy Bowers')
respuesta = sql_query('''SELECT value FROM solution''')

#No sabemos por qué ha dejado de funcionar esta línea de código, pero devolvía que la respuesta es correcta.
respuesta["value"][0]

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

<h2>¿Qué declaró Jeremy?

In [18]:
df = sql_query('''SELECT * FROM Interview WHERE person_id = 67318''')
df["transcript"][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

<h2>¿Quién tiene un Tesla Model S y es mujer?

In [19]:
df2 = sql_query('''SELECT * FROM drivers_license WHERE car_model = "Model S" AND gender = "female"''')
df2

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,682231,51,80,green,grey,female,3Z0M62,Tesla,Model S
3,918773,48,65,black,red,female,917UU3,Tesla,Model S


<h2>Filtramos por aquellas sospechosas con pelo rojo

In [20]:
mask = df2["hair_color"].str.contains("red")
df2[mask]

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
3,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [21]:
df = sql_query('''SELECT * FROM person where license_id = 202298''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [22]:
df = sql_query('''SELECT * FROM person where license_id = 291182''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,90700,Regina George,291182,332,Maple Ave,337169072


In [23]:
df = sql_query('''SELECT * FROM person where license_id = 918773''')
df

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910


<h2>Nos quedamos con 3 sospechosas:

* Miranda Priestly
* Regina George
* Red Korb

In [24]:
solution = (0, 'Miranda Priestly')
respuesta = sql_query('''SELECT value FROM solution''')

#No sabemos por qué ha dejado de funcionar esta línea de código, pero devolvía que la respuesta es correcta.
respuesta["value"][0]

,value


Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!